In [ ]:
'''
load logistic model

get bleu score on validation set
'''
logistic_decoder = LogisticDecoder(12, get_tokenizer_from_samples(small_train_samples))
logistic_decoder.load('../models/compare_3_models/logistic_model')



In [ ]:
'''
load RNN without dropout

get bleu score on validation set
'''

In [ ]:
'''
load RNN with dropout layers

get bleu score on validation set
'''

In [ ]:
'''
Note:

best model was ______

now do hyperparameter tuning on best model
'''